In [1]:
import json

import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
import os
from bert_util import bert_tokenize_data, tensor_train_test_split, train_bert_model, model_predict, get_data_loader, \
    calculate_accuracy
from util import get_dataframe_from_json
from sklearn.metrics import classification_report

os.environ["USE_TF"] = "0"

/home/emiliano/Desktop/Università/Human Language Technologies/Project/HLT Project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-21 18:42:14.005978: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 18:42:14.013432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747845734.021803   15590 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered


In [3]:
train_df = pd.read_json('../train_dataset.json', lines=False)
test_df = pd.read_json('../test_dataset.json', lines=False)
train_df

,turn,utterance,emotion,act,hat
0,3,"I'll take one, too.",happiness,inform,0
1,8,"You know, we are superior to other clothes com...",no_emotion,inform,3
2,5,"Her new boyfriend, right?",no_emotion,commissive,1
3,9,How about recommending him to use the storage ...,no_emotion,directive,4
4,1,"Oh, a bouquet of flowers. It's very kind of you.",surprise,commissive,1
...,...,...,...,...,...
808,0,I prefer potatoes to eggplants.,no_emotion,inform,0
809,0,"Mr. Smith, I would like to get right to the po...",no_emotion,question,1
810,4,Yeah?,no_emotion,question,1
811,0,I am so bored all day.,no_emotion,inform,0


In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
token_ids, attention_masks = bert_tokenize_data(tokenizer, train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(train_df['hat'].values), token_ids, attention_masks, test_size=0.1)

In [5]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()
num_training_steps = epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, epochs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-------------------- Epoch 1 --------------------

Training:
---------
Start Time:       2025-05-21 18:47:09.612088
Average Loss:     1.2928701347630958
Time Taken:       0:00:10.870746

Validation:
---------
Start Time:       2025-05-21 18:47:20.483301
Average Loss:     1.348430503498424
Average Accuracy: 0.48863636363636365
Time Taken:       0:00:00.375991

-------------------- Epoch 2 --------------------

Training:
---------
Start Time:       2025-05-21 18:47:20.859794
Average Loss:     1.068808484012666
Time Taken:       0:00:10.467976

Validation:
---------
Start Time:       2025-05-21 18:47:31.328147
Average Loss:     1.4532366665926846
Average Accuracy: 0.5
Time Taken:       0:00:00.382272

-------------------- Epoch 3 --------------------

Training:
---------
Start Time:       2025-05-21 18:47:31.710835
Average Loss:     0.8163206653426522
Time Taken:       0:00:10.477185

Validation:
---------
Start Time:       2025-05-21 18:47:42.188428
Average Loss:     1.3265056691386483
A

In [6]:
test_texts = test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = test_df['hat'].values.flatten()
accuracy = np.sum(preds == labels_flat) / len(labels_flat)
accuracy

np.float64(0.5735294117647058)

In [7]:
hat_map = {
    0: "red",
    1: "white",
    2: "black",
    3: "yellow",
    4: "green",
}
preds_array = np.array(preds)
print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

              precision    recall  f1-score   support

         red       0.46      0.40      0.43        40
       white       0.69      0.75      0.72       110
       black       0.38      0.39      0.38        23
      yellow       0.64      0.37      0.47        19
       green       0.19      0.25      0.21        12

    accuracy                           0.57       204
   macro avg       0.47      0.43      0.44       204
weighted avg       0.58      0.57      0.57       204



In [9]:
augmented_train_df = pd.read_json('../eda_train_dataset.json', lines=False)

In [10]:


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
token_ids, attention_masks = bert_tokenize_data(tokenizer, augmented_train_df['utterance'].values)
train_dataloader, val_dataloader = tensor_train_test_split(torch.tensor(augmented_train_df['hat'].values), token_ids, attention_masks, test_size=0.1)
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
epochs = 10
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()
num_training_steps = epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

model = train_bert_model(model, optimizer, scheduler, train_dataloader, val_dataloader, epochs)
test_texts = test_df['utterance'].values
serie = pd.Series(test_texts)
tids, amids = bert_tokenize_data(tokenizer, serie, max_length=64)
dl = get_data_loader(tids, amids, batch_size=5, shuffle=False)
preds, confidences = model_predict(model, dl)
labels_flat = test_df['hat'].values.flatten()

preds_array = np.array(preds)
print(classification_report(labels_flat, preds_array, target_names=list(hat_map.values())))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


-------------------- Epoch 1 --------------------

Training:
---------
Start Time:       2025-05-21 18:49:45.599930
Average Loss:     0.8352723453822158
Time Taken:       0:00:49.121369

Validation:
---------
Start Time:       2025-05-21 18:50:34.721732
Average Loss:     1.6427994484597064
Average Accuracy: 0.4376899696048632
Time Taken:       0:00:01.687571

-------------------- Epoch 2 --------------------

Training:
---------
Start Time:       2025-05-21 18:50:36.409734
Average Loss:     0.19497602208099393
Time Taken:       0:00:49.416321

Validation:
---------
Start Time:       2025-05-21 18:51:25.826465
Average Loss:     1.0135578555054963
Average Accuracy: 0.7815349544072948
Time Taken:       0:00:01.698944

-------------------- Epoch 3 --------------------

Training:
---------
Start Time:       2025-05-21 18:51:27.525819
Average Loss:     0.07621283072581728
Time Taken:       0:00:49.632151

Validation:
---------
Start Time:       2025-05-21 18:52:17.158519
Average Loss:     1.